In [32]:
!pip install -q efficientnet

In [33]:
!pip install optuna

In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os 
import random
import math
from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow.keras.backend as K
#import efficientnet.tfkeras as efn
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L
import efficientnet.tfkeras as efn
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as L
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Conv2D 
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense 
from tensorflow.keras import Model
import tensorflow.keras.optimizers as optim


from keras.utils.np_utils import to_categorical

import optuna

In [35]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

In [36]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [37]:
# Set Random Seed
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")
set_seed()

Random seed set as 42


In [38]:
# Configuration
IMAGE_SIZE = 800
BATCH_SIZE = 16 

# 1.Preprocessing

In [39]:
raw_dir = '/kaggle/input/flower-image-dataset/flowers'
paths = []
labels = []
for i in os.listdir(raw_dir):
    paths.append(os.path.join(raw_dir,i))
    labels.append(i.split('_')[0])

raw_input = pd.DataFrame({'path':paths,
                          'label':labels})
raw_input.head()

,path,label
0,/kaggle/input/flower-image-dataset/flowers/gar...,gardenias
1,/kaggle/input/flower-image-dataset/flowers/peo...,peonies
2,/kaggle/input/flower-image-dataset/flowers/peo...,peonies
3,/kaggle/input/flower-image-dataset/flowers/orc...,orchids
4,/kaggle/input/flower-image-dataset/flowers/gar...,gardenias


In [40]:
# Convert label to Int category
LE = LabelEncoder()
raw_input['category'] = LE.fit_transform(raw_input['label'])

# 2.Train test split

In [41]:
# Train test split
X = raw_input['path'].values
y = to_categorical(raw_input['category'], num_classes = 10)
X_2, X_test, y_2, y_test = train_test_split(X, y, test_size=0.095, random_state=42,shuffle = True, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_2, y_2, test_size=0.2, random_state=42,shuffle = True, stratify = y_2)

In [42]:
print(f'X_train has {len(X_train)} images; X_val has {len(X_val)} images; X_test has {len(X_test)} images')

X_train has 530 images; X_val has 133 images; X_test has 70 images


# 3.Prepare Tensorflow Dataset

In [43]:
# Prepare Data

def load_dataset(filepath,label):
    dataset = tf.data.Dataset.from_tensor_slices((filepath, label))
    return dataset


def decode_image(filepath, label=None):
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label

## 3.1.Rotation Transformation 

In [44]:
# Get idea from https://www.kaggle.com/code/cdeotte/rotation-augmentation-gpu-tpu-0-96

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

In [45]:
def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]),label

## 3.2.Prepare Train/Test/Valid Dataset

In [46]:
def get_training_dataset(dataset,do_aug=True):
    dataset = dataset.map(decode_image)
    dataset = dataset.cache()
    dataset = dataset.map(data_augment)
    if do_aug: 
        dataset = dataset.map(transform)
    #dataset = dataset.shuffle(128)
    dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
    return dataset

def get_validation_dataset(dataset):
    dataset = dataset.map(decode_image)
    dataset = dataset.cache()
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def get_test_dataset(dataset):
    dataset = dataset.map(decode_image)
    dataset = dataset.cache()
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

## 3.3.Get Train/Test/Valid Dataset

In [47]:
train_dataset = get_training_dataset(load_dataset(X_train, y_train),do_aug=True)

valid_dataset = get_validation_dataset(load_dataset(X_val, y_val))

test_dataset =  get_test_dataset(load_dataset(X_test, y_test))

# 4.Build Model and Hyper Parameter Tuning

In [48]:
efnet = efn.EfficientNetB7(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    weights='noisy-student',
    include_top=False,
    pooling='avg')
efnet.trainable = False

In [49]:
def build_model(trial):
    
    n_layers = trial.suggest_int("n_layers", 0, 3)
    layers = []
    layers.append(efnet)
  
    for i in range(n_layers):
        
        # Tune the number of dense layers after efnet
        dense_units = trial.suggest_categorical(f"dense_units_l{i+1}", [128, 256, 512, 768])
        
        layers.append(tf.keras.layers.Dense(dense_units, activation='relu'))

        
    layers.append(tf.keras.layers.Dense(10, activation='softmax'))
    
    return tf.keras.Sequential(layers)

In [50]:
def train_and_evaluate(param, model):
    
    model.compile(
                optimizer = getattr(optim, param['optimizer'])(learning_rate = param['learning_rate']),
                loss = 'categorical_crossentropy',
                metrics=['categorical_accuracy']
              )
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)
    history = model.fit(train_dataset,validation_data=valid_dataset,epochs=30, callbacks = [early_stopping],verbose=0)
    
    validset_acc = np.max(history.history['val_categorical_accuracy'])
    
    return validset_acc
    

In [51]:
def objective(trial):

    params = {
          'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
          'learning_rate': trial.suggest_categorical('learning_rate', [1e-5, 5e-4, 1e-4, 5e-3, 1e-3, 5e-2, 1e-1])
          }

    model = build_model(trial)

    accuracy = train_and_evaluate(params, model)

    return accuracy

In [52]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())

[I 2023-03-31 17:09:53,149] A new study created in memory with name: no-name-2dba0c9a-3126-487b-8a08-c3a39a5dedba


In [53]:
study.optimize(objective, n_trials=3)

2023-03-31 17:10:10.064047: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_5/efficientnet-b7/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
[I 2023-03-31 17:44:58,234] Trial 0 finished with value: 0.11278195679187775 and parameters: {'optimizer': 'SGD', 'learning_rate': 0.0001, 'n_layers': 3, 'dense_units_l1': 128, 'dense_units_l2': 128, 'dense_units_l3': 256}. Best is trial 0 with value: 0.11278195679187775.
2023-03-31 17:45:14.932607: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_6/efficientnet-b7/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
[I 2023-03-31 17:58:25,278] Trial 1 finished with value: 0.9172932505607605 and parameters: {'optimizer': 'RMSprop', 'learning_rate': 0.005, 'n_layer

# 5.Show Tuning Results

In [54]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

optimizer: RMSprop
learning_rate: 0.005
n_layers: 1
dense_units_l1: 512


In [58]:
optuna.visualization.plot_param_importances(study)

In [59]:
optuna.visualization.plot_optimization_history(study)

In [60]:
optuna.visualization.plot_parallel_coordinate(study)